# Review Sentiment & Summary Tool
# Review Helper — Sentiment, Summary & Rewrite

This notebook turns short reviews into something easier to work with:
- **Sentiment**: positive / neutral / negative (with a one-line rationale)
- **Summary**: a factual 1–2 line takeaway
- **Rewrite**: a clearer version of the original review

**Why**: Teams often need to quickly scan user feedback to spot patterns. This helps reduce noise and standardize text for dashboards, triage, or further analysis.

**How it could be used**:
- Auto-summarize new product reviews before sending to a support queue
- Clean up messy feedback before storing in a data warehouse
- Plug into a lightweight Streamlit app for non-technical users


## Setup
We start with imports and API key setup. The API key should be stored securely as an environment variable.

In [1]:
# --- Setup (short) ---
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()  # reads the .env file next to this notebook

api_key = (os.getenv("OPENAI_API_KEY") or "").strip()
assert api_key, "No OPENAI_API_KEY found. Did you create the .env file in THIS folder?"

client = OpenAI(api_key=api_key)
print("✅ client ready")


✅ client ready


In [3]:
# --- Test that the env var is visible (does not print the key) ---
import os
assert os.getenv("OPENAI_API_KEY"), "No OPENAI_API_KEY found. Is your .env beside the notebook?"
print("✅ .env loaded correctly")


✅ .env loaded correctly


In [4]:
try:
    r = client.responses.create(
        model="gpt-4o-mini",
        input="Say 'ready' in one word."
    )
    print("Model replied:", r.output_text)
except Exception as e:
    # Show a readable error if auth fails
    import json, re
    msg = str(e)
    # Shorten the noise a little
    msg = re.sub(r"\s+", " ", msg).strip()
    print("Smoke test failed.\n", msg)


✅ Model replied: Ready.


## Prompt Setup
We define three simple prompts that are short, explicit, and testable.

In [3]:
# --- Prompt Setup ---

SENTIMENT_PROMPT = """You are an analyst. Determine the sentiment of the text as one of:
Positive, Neutral, or Negative. Return JSON with fields: sentiment, rationale (one sentence).
Text: {review}"""

SUMMARY_PROMPT = """You are a helpful assistant. Summarize the core message of the review
in 1–2 concise lines. Avoid opinionated words. Focus on facts (battery life, build quality, etc.).
Text: {review}"""

REWRITE_PROMPT = """Rewrite the review to be clear, grammatical, and neutral in tone.
Keep 1 short paragraph. Keep product-specific details. Do not change the meaning.
Text: {review}"""


## Helper Function
We define a helper function to call the API in a consistent way.

In [4]:
# --- Helper Function ---
def call_llm(prompt: str, model: str = "gpt-4o-mini") -> str:
    try:
        resp = client.responses.create(model=model, input=prompt)
        return resp.output_text
    except Exception as e:
        return f"Error: {e}"


## Analyze Review Function
This function runs sentiment, summary, and rewrite for each review.

In [5]:
# --- Analyze Review Function ---
from typing import Dict

def analyze_review(review: str, model: str = "gpt-4o-mini") -> Dict[str, str]:
    sentiment_json = call_llm(SENTIMENT_PROMPT.format(review=review), model)
    summary       = call_llm(SUMMARY_PROMPT.format(review=review), model)
    rewrite       = call_llm(REWRITE_PROMPT.format(review=review), model)
    return {
        "review": review,
        "sentiment_json": sentiment_json,
        "summary": summary,
        "rewrite": rewrite,
    }


## Sample Reviews
We’ll start with a few quick examples. You can replace these with your own dataset later.

In [6]:
# --- Sample Reviews ---
sample_reviews = [
    "The laptop is okay, but the battery dies in three hours and the screen is dim in sunlight.",
    "Absolutely love this phone—camera quality is insane and the battery easily lasts two days!",
    "Delivery was late. Product works as expected though, so it's fine."
]


## Run the Tool
We now apply our function to the sample reviews.

In [12]:
# --- Run the Tool ---
import pandas as pd
pd.set_option("display.max_colwidth", 80)  # keeps the table compact for screenshots

results = [analyze_review(r) for r in sample_reviews]
df = pd.DataFrame(results, columns=["review", "sentiment_json", "summary", "rewrite"])
df.head(3)



,review,sentiment_json,summary,rewrite
0,"The laptop is okay, but the battery dies in three hours and the screen is di...","```json\n{\n ""sentiment"": ""Negative"",\n ""rationale"": ""The text conveys dis...",The laptop has a battery life of three hours and the screen visibility is po...,"The laptop functions adequately, but the battery life is limited to three ho..."
1,Absolutely love this phone—camera quality is insane and the battery easily l...,"```json\n{\n ""sentiment"": ""Positive"",\n ""rationale"": ""The text expresses s...",The phone features excellent camera quality and a battery life of up to two ...,"I am very impressed with this phone; the camera quality is excellent, and th..."
2,"Delivery was late. Product works as expected though, so it's fine.","```json\n{\n ""sentiment"": ""Neutral"",\n ""rationale"": ""While the delivery wa...","The product functions as intended, but delivery was delayed.","The delivery was delayed; however, the product functions as expected, so ove..."


## Findings
The sentiment analysis works well and gives structured JSON that’s easy to work with.

Summaries stay short and to the point, focusing on the main facts instead of opinions.

The rewrite step makes reviews clearer and more readable without changing the meaning.

The tool runs quickly with gpt-4o-mini, and switching to a stronger model like gpt-4.1 could improve accuracy on harder cases.

It handles normal reviews smoothly, but it would be worth testing trickier cases like sarcasm, mixed opinions, or emotional wording to see how robust it is.

## Next Steps
Some ways to expand this project:
- Run the tool on a larger dataset of reviews (CSV or scraped data).
- Add evaluation metrics to compare summaries against ground truth.
- Fine-tune prompt wording for more consistency.
- Integrate into a web app with Streamlit or Flask.